### Show Volskew calculation

## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from IPython.core.display import  HTML
from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin,Modebar
from IPython import display

import datetime
from dateutil.relativedelta import relativedelta
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb,pdb
import traceback
import pandas_datareader.data as pdr
from scipy.stats import norm

from ipysheet import from_dataframe,to_dataframe
from dashapp import dashapp2 as dashapp
# import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px



#### Step 02: define important functions that are used below

In [3]:
def dt_to_yyyymmdd(d):
    return int(d.year)*100*100 + int(d.month)*100 + int(d.day)

def str_to_yyyymmdd(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    s = '%04d%02d%02d' %(dt.year,dt.month,dt.day)
    return int(s)

def str_to_date(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    return dt


def fetch_history(symbol,dt_beg,dt_end):
    df = pdr.DataReader(symbol, 'yahoo', dt_beg, dt_end)
    # move index to date column, sort and recreate index
    df['date'] = df.index
    df = df.sort_values('date')
    df.index = list(range(len(df)))
    # make adj close the close
    df = df.drop(['Adj Close'],axis=1)
    cols = df.columns.values 
    cols_dict = {c:c[0].lower() + c[1:] for c in cols}
    df = df.rename(columns = cols_dict)
    df['settle_date'] = df.date.apply(str_to_yyyymmdd)
    return df


### Create a dataframe that holds a "base" volskew, representing a volskew with:
1. the median S&P vol skew 10% below the money, for options on the CME ES contract from 2011 to 2020
2. the vol skew of the rows from number 1, that has the median skew 10% above the money

In [4]:
df_iv_skew_ES = pd.read_csv('iv_skew_ES_2011_2020.csv')

df_iv_skew_ES2 = df_iv_skew_ES[df_iv_skew_ES['-0.1'] == df_iv_skew_ES['-0.1'].median()]
df_iv_skew_ES2 = df_iv_skew_ES2[df_iv_skew_ES2['0.1'] == df_iv_skew_ES2['0.1'].median()]
df_only_strikes = df_iv_skew_ES2.iloc[0:1][[c for c in df_iv_skew_ES2.columns.values if '.' in c]]
strikes = [round(float(c),2) for c in  df_only_strikes.columns.values]
skews = df_only_strikes.iloc[0].values
df_skew = pd.DataFrame({'strike':strikes,'vol_skew':skews})
df_skew.index = df_skew.strike
df_skew = df_skew[['vol_skew']]
def _calc_skew_from_df_skew(underlying_price,strike):
    curr_perc_otm = round(strike/underlying_price - 1,5)
    df_skew2 = df_skew.copy()
    row_to_append = df_skew2.iloc[-1]
    row_to_append.name = curr_perc_otm
    row_to_append['vol_skew'] = np.nan
    df_skew2 = df_skew2.append(row_to_append)
    df_skew2 = df_skew2.sort_index()
    df_skew2['vol_skew'] = df_skew2.vol_skew.interpolate(method='polynomial', order=2)
    skew = df_skew2.loc[curr_perc_otm,'vol_skew']
    return skew




In [5]:
# test it - value should be about 0.0653434
_calc_skew_from_df_skew(2900,2900*(1-.12345))


0.065343393830266

## END